In [1]:
using LorentzianSimplexSolver;

    CondaPkg Found dependencies: /Users/xiaoquer/.julia/packages/PythonCall/83z4q/CondaPkg.toml
    CondaPkg Found dependencies: /Users/xiaoquer/.julia/packages/CondaPkg/0UqYV/CondaPkg.toml
    CondaPkg Found dependencies: /Users/xiaoquer/.julia/environments/v1.11/CondaPkg.toml
    CondaPkg Initialising pixi
             │ /Users/xiaoquer/.julia/artifacts/cc02fad3b04cfb688ba4e3d60648ad8c3afe3a2d/bin/pixi
             │ init
             │ --format pixi
             └ /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg
✔ Created /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg/pixi.toml
    CondaPkg Wrote /Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/LorentzianSimplexSolver/.CondaPkg/pixi.toml
             │ [dependencies]
             │ numpy = "*"
             │ sympy

In [2]:
# ------------------------------------------------------------
# 1. Precision choice (user-controlled)
# ------------------------------------------------------------
const ScalarT = Float64
# const ScalarT = BigFloat

if ScalarT === BigFloat
    LorentzianSimplexSolver.PrecisionUtils.set_big_precision!(prec)
    LorentzianSimplexSolver.PrecisionUtils.set_tolerance!(sqrt(eps(BigFloat)))
else
    LorentzianSimplexSolver.PrecisionUtils.set_tolerance!(1e-10)
end

# ------------------------------------------------------------
# 2. Read simplices
# ------------------------------------------------------------
simplices = [[1,2,3,4,5], [1,2,4,5,6]]

ns = length(simplices)

all_vertices = unique(Iterators.flatten(simplices))
sort!(all_vertices)

Nverts = length(all_vertices)

# ------------------------------------------------------------
# 3. Read vertex coordinates
# ------------------------------------------------------------
vertex_coords = Dict{Int, Vector{ScalarT}}()    

coords_lines = [
    "0, 0, 0, 0",
    "0, 0, 0, 1",
    "0, 0, 1, 1",
    "0, 1, 1, 1",
    "1//2, 1, 1, 1",
    "0, 1, 0, 1",
]

for (v, line) in zip(all_vertices, coords_lines)
    vertex_coords[v] = LorentzianSimplexSolver.PrecisionUtils.parse_numeric_line(line, ScalarT)
end

In [3]:
# ------------------------------------------------------------
# 4. Build geometry
# ------------------------------------------------------------
datasets = LorentzianSimplexSolver.GeometryTypes.GeometryDataset{ScalarT}[]

for (s, simplex) in enumerate(simplices)
    println("\n--- Processing simplex $s with vertices $simplex ---")
    bdypoints = [vertex_coords[v] for v in simplex]
    ds = LorentzianSimplexSolver.GeometryPipeline.run_geometry_pipeline(bdypoints)
    push!(datasets, ds)
end

geom_base = LorentzianSimplexSolver.GeometryTypes.GeometryCollection(datasets);


--- Processing simplex 1 with vertices [1, 2, 3, 4, 5] ---

--- Processing simplex 2 with vertices [1, 2, 4, 5, 6] ---


In [4]:
# ------------------------------------------------------------
# 5. Consistency checks for each simplex
# ------------------------------------------------------------
for (idx, simplex) in enumerate(geom_base.simplex)
    LorentzianSimplexSolver.GeometryConsistency.check_sl2c_parallel_transport(simplex.solgsl2c, simplex.bdybivec55)
    LorentzianSimplexSolver.GeometryConsistency.check_so13_parallel_transport(simplex.solgso13, simplex.bdybivec4d55)
    LorentzianSimplexSolver.GeometryConsistency.check_closure_bivectors(simplex.kappa, simplex.areas, simplex.bdybivec55)
end

✓ SL(2,C) parallel transport satisfied (max residual = 2.7755575615628914e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 5.266250202865052e-16)
✓ Closure condition satisfied for bivectors (max residual = 8.777083671441753e-17)
✓ SL(2,C) parallel transport satisfied (max residual = 2.7755575615628914e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 5.266250202865052e-16)
✓ Closure condition satisfied for bivectors (max residual = 8.777083671441753e-17)


In [5]:
# ------------------------------------------------------------
# 6. Connect simplices + face matching + gauge fixing
# ------------------------------------------------------------
if ns > 1
    # println("\nFixing global κ-sign orientation ...")
    LorentzianSimplexSolver.KappaOrientation.fix_kappa_signs!(simplices, geom_base)

    # println("\nBuilding global connectivity ...")
    conn = LorentzianSimplexSolver.FourSimplexConnectivity.build_global_connectivity(simplices, geom_base)
    push!(geom_base.connectivity, conn)

    geom_ref    = deepcopy(geom_base)
    geom_parity = deepcopy(geom_base)
    LorentzianSimplexSolver.FaceXiMatching.run_face_xi_matching(geom_ref; sector=:ref)
    LorentzianSimplexSolver.FaceXiMatching.run_face_xi_matching(geom_parity; sector=:parity)
    # println("Global connectivity constructed for both relevant orientations.")

    # println("\nPerform SU(2) and SU(1,1) gauge fixing ...")
    LorentzianSimplexSolver.GaugeFixingSU.run_su2_su11_gauge_fix(geom_ref)
    LorentzianSimplexSolver.GaugeFixingSU.run_su2_su11_gauge_fix(geom_parity)
    # println("\nGauge fixing finished.")
end

  SL(2,C) matrices updated:      ✓
  SL(2,C) parity matrices updated: ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓

  SL(2,C) matrices updated:      ✓
  SL(2,C) parity matrices updated: ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓



In [6]:
# ------------------------------------------------------------
# 7a. Symbols and action (reference orientation)
# ------------------------------------------------------------
LorentzianSimplexSolver.DefineSymbols.run_define_variables(geom_ref)

sd_ref, _ = LorentzianSimplexSolver.SolveVars.run_solver(geom_ref)

S_ref = LorentzianSimplexSolver.DefineAction.compute_action(geom_ref)

S_ref_fn, labels_ref =
    LorentzianSimplexSolver.SymbolicToJulia.build_action_function(S_ref, sd_ref)

using Symbolics
@variables γ
args_ref =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector(sd_ref, labels_ref, γ)
args_ref_keep_j =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector_keep_j(sd_ref, labels_ref, γ)

S_ref_sym = expand(simplify(S_ref_fn(args_ref...)))
S_ref_sym_keep_j = expand(simplify(S_ref_fn(args_ref_keep_j...)));

In [7]:
# ------------------------------------------------------------
# 7b. Symbols and action (parity orientation)
# ------------------------------------------------------------
LorentzianSimplexSolver.DefineSymbols.run_define_variables(geom_parity)

sd_parity, _ = LorentzianSimplexSolver.SolveVars.run_solver(geom_parity)

S_parity = LorentzianSimplexSolver.DefineAction.compute_action(geom_parity)

S_parity_fn, labels_parity =
    LorentzianSimplexSolver.SymbolicToJulia.build_action_function(S_parity, sd_parity)

args_parity =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector(sd_parity, labels_parity, γ)
args_parity_keep_j =
    LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector_keep_j(sd_parity, labels_parity, γ)

S_parity_sym = expand(simplify(S_parity_fn(args_parity...)))
S_parity_sym_keep_j = expand(simplify(S_ref_fn(args_parity_keep_j...)));

In [8]:
S_regge_num,  S_regge_symbolics = LorentzianSimplexSolver.ReggeAction.run_Regge_action(geom_ref, γ);

In [9]:
orientation = LorentzianSimplexSolver.OrientationSelector.select_orientation(S_ref_sym_keep_j, S_parity_sym_keep_j, S_regge_symbolics, γ)

phase = (S_ref_sym + S_parity_sym)//2

if orientation == :ref_neg || orientation == :parity_pos
    S_pos = expand(simplify(S_parity_sym - phase))
    S_neg = expand(simplify(S_ref_sym - phase))
else
    S_pos = expand(simplify(S_ref_sym - phase))
    S_neg = expand(simplify(S_parity_sym - phase))
end

println("Action at the positive-orientation critical point: $S_pos")
println("Action at the negative-orientation critical point: $S_neg")

Action at the positive-orientation critical point: (im*(-0.33372368702131283 - 1.0725563700114555γ)) / γ
Action at the negative-orientation critical point: (im*(0.33372368702131283 + 1.0725563700114555γ)) / γ
